In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error
import folium
from geopy.distance import geodesic


#Loading and preprocess the location data
file_path = "C:\\Users\\ADMIN\\Downloads\\location_history_varying_with_day.csv"
df = pd.read_csv(file_path)

# Parse timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce')

# Handle null values
df.dropna(subset=['accuracy', 'latitude', 'longitude', 'distance'], inplace=True)

# Filter out high accuracy values
max_accuracy_threshold = 100  # Example threshold for maximum acceptable accuracy
df = df[df['accuracy'] <= max_accuracy_threshold]

# Feature engineering + adding hour, isweekend
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['hour'] = df['timestamp'].dt.hour
df['month'] = df['timestamp'].dt.month
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)  # 0 for weekday, 1 for weekend

def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# Apply to your dataframe
df = remove_outliers(df, ['latitude', 'longitude'])
# Normalize features
scaler = MinMaxScaler()
df[['day_of_week', 'hour', 'month', 'is_weekend']] = scaler.fit_transform(df[['day_of_week', 'hour', 'month', 'is_weekend']])

# Normalize latitude and longitude
df['normalized_latitude'] = (df['latitude'] - df['latitude'].mean()) / df['latitude'].std()
df['normalized_longitude'] = (df['longitude'] - df['longitude'].mean()) / df['longitude'].std()

# Prepare input sequences !!
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data)-seq_length):
        x = data.iloc[i:i+seq_length][['normalized_latitude', 'normalized_longitude', 'day_of_week', 'hour', 'month', 'is_weekend']].values
        y = data.iloc[i+seq_length][['normalized_latitude', 'normalized_longitude']].values
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

sequence_length = 10  # Example sequence length
X, y = create_sequences(df, sequence_length)

#Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Ensure data type compatibility
X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_val = y_val.astype(np.float32)
y_test = y_test.astype(np.float32)

# LSTM MODEL DEFINE AND TRAIN
model = Sequential([
    LSTM(units=64, activation='relu', input_shape=(sequence_length, 6)),  # 6 input features now
    Dropout(0.2),
    Dense(units=32, activation='relu'),
    Dense(units=2)
])

model.compile(optimizer='adam', loss='mse')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Model evaluation
loss = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")

# Predictionsss
predictions = model.predict(X_test)

# Denormalize predictions and actual test data
predictions[:, 0] = predictions[:, 0] * df['latitude'].std() + df['latitude'].mean()
predictions[:, 1] = predictions[:, 1] * df['longitude'].std() + df['longitude'].mean()

y_test_denorm = y_test.copy()
y_test_denorm[:, 0] = y_test_denorm[:, 0] * df['latitude'].std() + df['latitude'].mean()
y_test_denorm[:, 1] = y_test_denorm[:, 1] * df['longitude'].std() + df['longitude'].mean()

# Calculate errors
mae = mean_absolute_error(y_test_denorm, predictions)
rmse = np.sqrt(mean_squared_error(y_test_denorm, predictions))
print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")


def calculate_distance(coord1, coord2):
    """Calculate the distance between two coordinates in kilometers."""
    return geodesic(coord1, coord2).kilometers

def check_location_alarm(actual, predicted, threshold_km=1.0):
    """Check if the predicted location is too far from the actual location."""
    distance = calculate_distance(actual, predicted)
    if distance > threshold_km:
        print(f"ALARM: Predicted location is {distance:.2f} km away from actual location!")
        return True
    return False

def visualize_predictions(actual, predicted, n_points=100, alarm_threshold_km=1.0):
    center_lat, center_lon = np.mean(actual, axis=0)
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
    
    for i in range(min(n_points, len(actual))):
        actual_location = actual[i]
        predicted_location = predicted[i]
        
        # Check for alarm
        alarm_triggered = check_location_alarm(actual_location, predicted_location, alarm_threshold_km)
        
        # Actual location marker
        folium.CircleMarker(
            location=actual_location,
            radius=5,
            popup=f"Actual {i}",
            color="blue",
            fill=True
        ).add_to(m)
        
        # Predicted location marker
        folium.CircleMarker(
            location=predicted_location,
            radius=5,
            popup=f"Predicted {i}",
            color="red" if alarm_triggered else "green",
            fill=True
        ).add_to(m)
        
        # Draw a line between actual and predicted if alarm is triggered
        if alarm_triggered:
            folium.PolyLine(locations=[actual_location, predicted_location], color="red", weight=2, opacity=0.8).add_to(m)
    
    return m

# Simulate current location tracking
n_visualize = 100
current_locations = y_test_denorm[:n_visualize]  # Using test data as "current" locations
predicted_locations = predictions[:n_visualize]

# Visualize with alarm feature
alarm_threshold_km = 1.0  # Set your desired threshold here
map_viz = visualize_predictions(current_locations, predicted_locations, n_visualize, alarm_threshold_km)
map_viz.save("predictions_map_with_alarm.html")

print("Map visualization with alarm feature saved as 'predictions_map_with_alarm.html'")

# Demonstrate alarm functionality
for i in range(n_visualize):
    actual = current_locations[i]
    predicted = predicted_locations[i]
    check_location_alarm(actual, predicted, alarm_threshold_km)

ModuleNotFoundError: No module named 'tensorflow'

In [9]:
import folium

def visualize_predictions(actual, predicted, n_points=100):
    # Create a map centered on the mean of actual locations
    center_lat, center_lon = np.mean(actual, axis=0)
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
    
    # Plot actual and predicted points
    for i in range(min(n_points, len(actual))):
        folium.CircleMarker(
            location=actual[i],
            radius=5,
            popup=f"Actual {i}",
            color="blue",
            fill=True
        ).add_to(m)
        
        folium.CircleMarker(
            location=predicted[i],
            radius=5,
            popup=f"Predicted {i}",
            color="red",
            fill=True
        ).add_to(m)
    
    return m

# Visualize a subset of test data
n_visualize = 100
map_viz = visualize_predictions(y_test, predictions[:n_visualize])
map_viz.save("predictions_map3.html")

print("Map visualization saved as 'predictions_map.html'")

Map visualization saved as 'predictions_map.html'
